<a href="https://colab.research.google.com/github/crazyandiee/Customer-Churn-Prediction-System-with-Dashboard-Web-App/blob/main/notebooks/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from google.colab import files
uploaded = files.upload()  # upload the CSV file manually


In [21]:
df=pd.read_csv('/content/Data/Raw/WA_Fn-UseC_-Telco-Customer-Churn.csv')

print("\n--- Basic Info ---")
print(df.info())
print("\n--- Missing Values ---")
print(df.isna().sum())
print("\n--- Sample Rows ---")
print(df.head(3))


--- Basic Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043

In [22]:
# Clean Data Types
# 'TotalCharges' sometimes comes as string/object with blanks — convert to numeric
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Check how many were converted to NaN
print("\nMissing values in TotalCharges after conversion:", df['TotalCharges'].isna().sum())

# Fill missing TotalCharges with median (you can also use mean)
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())



Missing values in TotalCharges after conversion: 11


In [23]:
# Drop Irrelevant Columns
# 'customerID' is just an identifier
if 'customerID' in df.columns:
    df.drop(columns=['customerID'], inplace=True)

In [24]:
# Only map if Churn exists and not yet numeric
if df['Churn'].dtype == 'object':
    df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

In [25]:
# Feature Engineering

# tenure group (categorical bucket)
df['tenure_group'] = pd.cut(df['tenure'],
                            bins=[-1, 3, 12, 24, 60, np.inf],
                            labels=['0-3', '4-12', '13-24', '25-60', '60+'])

# count number of "Yes" services
service_cols = [
    'PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
    'DeviceProtection','TechSupport','StreamingTV','StreamingMovies'
]
df['num_services'] = df[service_cols].apply(lambda row: sum(row == 'Yes'), axis=1)

# payment method flag
df['is_electronic_payment'] = (df['PaymentMethod'] == 'Electronic check').astype(int)

# month-to-month contract flag
df['is_month_to_month'] = (df['Contract'] == 'Month-to-month').astype(int)

# average charge per month (guard against divide by zero)
df['avg_charge_per_month'] = df['TotalCharges'] / df['tenure'].replace(0, 1)


In [26]:
# Handle Categorical Variables
# Identify categorical columns (excluding target)
cat_cols = [col for col in df.select_dtypes(include='object').columns if col != 'Churn']

# One-hot encode categorical variables
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

print("\n✅ One-hot encoding complete. New shape:", df.shape)


✅ One-hot encoding complete. New shape: (7043, 36)


In [27]:
# Final Checks
print("\nFinal Data Overview:")
print(df.info())
print("\n--- Sample of Clean Data ---")
print(df.head(5))


Final Data Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 36 columns):
 #   Column                                 Non-Null Count  Dtype   
---  ------                                 --------------  -----   
 0   SeniorCitizen                          7043 non-null   int64   
 1   tenure                                 7043 non-null   int64   
 2   MonthlyCharges                         7043 non-null   float64 
 3   TotalCharges                           7043 non-null   float64 
 4   Churn                                  7043 non-null   int64   
 5   tenure_group                           7043 non-null   category
 6   num_services                           7043 non-null   int64   
 7   is_electronic_payment                  7043 non-null   int64   
 8   is_month_to_month                      7043 non-null   int64   
 9   avg_charge_per_month                   7043 non-null   float64 
 10  gender_Male                           